In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vtbdatafusion2022/currency_rk.csv
/kaggle/input/vtbdatafusion2022/sample_submission.csv
/kaggle/input/vtbdatafusion2022/click_categories.csv
/kaggle/input/vtbdatafusion2022/train_matching.csv
/kaggle/input/vtbdatafusion2022/mcc_codes.csv
/kaggle/input/vtbdatafusion2022/clickstream.csv
/kaggle/input/vtbdatafusion2022/train.csv
/kaggle/input/vtbdatafusion2022/transactions.csv


# Работа с данными для тренеровки

##### для начала посмотрим на те данные для которых мы уже знаем есть ли вышка

In [2]:
train = pd.read_csv("../input/vtbdatafusion2022/train.csv")
print('train.csv - DONE!')

train.csv - DONE!


##### train.csv: Табличный файл с целевой переменной: меткой наличия у клиентов ВТБ высшего образования — 0 и 1. Разметка представлена не для всех клиентов

In [3]:
train.head() #вывод верха таблицы

,bank,higher_education
0,3755b59782464456bac1aec1a44e0db3,0.0
1,604a550439d644718ea6e1693fbf03dc,0.0
2,542d4776ebe5454fb8ab36f1c276fe0e,1.0
3,ee37fecea44d475ca030cde7ff7d545d,0.0
4,18617a1100f44a99b3a0772341fec3db,0.0


##### для начала найдем все транзакции для каждого из этих людей, а для этого мы загрузим еще несколько файлов. А так же соединим эти файлы, чтобы получить одну общую таблицы по всем этим людям

In [4]:
tra_val = pd.read_csv('../input/vtbdatafusion2022/train_matching.csv')
print('train_matching.csv - DONE!')

train_matching.csv - DONE!


##### train_matching.csv: Целевая переменная — табличный файл с точным соответствием между id клиентов в данных транзакций и кликстримов. Разметка представлена не для всех клиентов, присутствующих в данных транзакций и кликстрима

In [5]:
# введем новую переменную в которую сведем первые 2 таблицы.
#стоит отметить что будем соединять по 1й табл, чтобы не потерять порядок людей!
df_H_edu_all = train.merge(tra_val)
df_H_edu_all.head()

,bank,higher_education,rtk
0,604a550439d644718ea6e1693fbf03dc,0.0,fece9b32c7d84acb836c3b1271cfdcc9
1,542d4776ebe5454fb8ab36f1c276fe0e,1.0,38ac442698524d178f6ca361e0c603bb
2,ee37fecea44d475ca030cde7ff7d545d,0.0,839669a7588b42ee9e01a9f200e5f638
3,079f07153c0149d19f54646618452929,0.0,f9d8f9f57453493186964049c43da914
4,6dee55b3d7284d18bed3094ee3103812,1.0,6d6b702132d94a8692cba48f4e28e7b2


##### далее загрузим файл в котором содержатся ВСЕ транзакции
( это займет какое-то время, тк файл не самый маленький )

In [6]:
trans = pd.read_csv('../input/vtbdatafusion2022/transactions.csv')
print('transactions.csv - DONE!')

transactions.csv - DONE!


##### transactions.csv: Список транзакций

In [7]:
trans.head()

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,000932580e404dafbecd5916d4640938,5411,48,-361.07230,2020-08-03 08:05:23
1,000932580e404dafbecd5916d4640938,5499,48,-137.31398,2020-08-05 01:27:40
2,000932580e404dafbecd5916d4640938,5499,48,-138.84981,2020-08-05 03:28:11
3,000932580e404dafbecd5916d4640938,4829,48,-309.47653,2020-08-06 00:36:29
4,000932580e404dafbecd5916d4640938,5411,48,-133.47370,2020-08-09 00:30:13


теперь для людей из df_H_edu_all найдем все их траты и запишим их в отдельную таблицу, но для начала надо именить названия поля в таблице trans,  чтобы названия были одинаковыми.

In [8]:
trans.rename(columns = {'user_id': 'bank'}, inplace = True)
trans

,bank,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,000932580e404dafbecd5916d4640938,5411,48,-361.072300,2020-08-03 08:05:23
1,000932580e404dafbecd5916d4640938,5499,48,-137.313980,2020-08-05 01:27:40
2,000932580e404dafbecd5916d4640938,5499,48,-138.849810,2020-08-05 03:28:11
3,000932580e404dafbecd5916d4640938,4829,48,-309.476530,2020-08-06 00:36:29
4,000932580e404dafbecd5916d4640938,5411,48,-133.473700,2020-08-09 00:30:13
...,...,...,...,...,...
19821905,fffb051877b949f49bed6538393a06ae,5411,48,-40.727768,2021-07-31 05:33:03
19821906,fffb051877b949f49bed6538393a06ae,5411,48,-277.494350,2021-07-31 08:57:02
19821907,fffb051877b949f49bed6538393a06ae,5411,48,-138.097350,2021-07-31 08:59:33
19821908,fffb051877b949f49bed6538393a06ae,6011,48,-86.046776,2021-08-01 23:04:41


In [9]:
#объединяем 
df_tran_H_edu_all = df_H_edu_all.merge(trans)
df_tran_H_edu_all.head()

,bank,higher_education,rtk,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,604a550439d644718ea6e1693fbf03dc,0.0,fece9b32c7d84acb836c3b1271cfdcc9,5921,48,-32.840740,2020-08-03 09:37:40
1,604a550439d644718ea6e1693fbf03dc,0.0,fece9b32c7d84acb836c3b1271cfdcc9,5411,48,-221.986970,2020-08-03 09:59:39
2,604a550439d644718ea6e1693fbf03dc,0.0,fece9b32c7d84acb836c3b1271cfdcc9,5411,48,-21.282393,2020-08-03 10:58:54
3,604a550439d644718ea6e1693fbf03dc,0.0,fece9b32c7d84acb836c3b1271cfdcc9,5462,48,-100.272316,2020-08-04 09:30:49
4,604a550439d644718ea6e1693fbf03dc,0.0,fece9b32c7d84acb836c3b1271cfdcc9,5411,48,-447.390620,2020-08-04 09:40:23


в приличном обществе за то что вы увидите дальше я бы получил по лицу, Но пока что есть, то есть

In [12]:
jss =  df_tran_H_edu_all.shape
jss = jss[0]
print(jss)

5716192


In [ ]:
mon_div_all = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
mon_plu_all = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

mon_div_all_USD = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
mon_plu_all_USD = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

mon_div_all_EUR = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
mon_plu_all_EUR = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

mon_div_all_xz = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
mon_plu_all_xz = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

col_VO_all = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )

mcc_all = np.zeros(len(pd.unique(df_tran_H_edu_all['bank'])) )
i = 0
j = 0


mon_div_all[i] += float(df_tran_H_edu_all.transaction_amt[j])
mcc[i] += int(df_tran_H_edu_all.mcc[j])


for j in  range (jss-1):
    #подсчет процентов, чтобы знать сколько будет делать
    if j/(jss//100) == int(j/(jss//100)): 
        print(j/(jss//100),'%')
        
    h1 = str(df_tran_H_edu_all.bank[j] )
    h2 = str(df_tran_H_edu_all.bank[j+1])
    
    if h1 == h2: # сверяю что идентифик j и j+1 одинаковые и тогда считаю для разных валют
        
        m2 = float(df_tran_H_edu_all.transaction_amt[j+1]) #деньги (j+1)-го
        
        col_VO_all [i] += 1 #кол-во транзакций для каждого человека
        
        temp_mcc = int(df_tran_H_edu_all.mcc[j+1])
        #в этом промежутке самое интересное, а именно траты на авиа белеты и прочее
        if int(df_tran_H_edu_all.mcc[j+1])>3000: 
            if int(df_tran_H_edu_all.mcc[j+1]) < 4000:
                temp_mcc *= 2
        mcc[i] += temp_mcc #считаю  mcc общий mcc код для каждого
            
        # тут определяю какая валюта и было списание или пополнение денег
        if float(df_tran_H_edu_all.currency_rk[j+1]) == float(48): 
            
            if m2 > 0:
                mon_plu_all[i] +=m2
            else:
                mon_div_all[i] += m2
                
        elif float(df_tran_H_edu_all.currency_rk[j+1]) == float(50):
            
            if m2 > 0:
                mon_plu_all_USD[i] +=m2
            else:
                mon_div_all_EUR[i] += m2
                
        elif float(df_tran_H_edu_all.currency_rk[j+1]) == float(60):
            
            if m2 > 0:
                mon_plu_all_EUR[i] +=m2
            else:
                mon_div_all_EUR[i] += m2
                
        else:
            
            if m2 > 0:
                mon_plu_all_xz[i] +=m2
            else:
                mon_div_all_xz[i] += m2
            
    else:
        m2 = float(df_tran_H_edu_all.transaction_amt[j+1])
        
        if float(df_tran_H_edu_all.currency_rk[j+1]) == float(48): #
            
            if m2 > 0:
                mon_plu_all[i] +=m2
            else:
                mon_div_all[i] += m2
                
        elif float(df_tran_H_edu_all.currency_rk[j+1]) == float(50):
            
            if m2 > 0:
                mon_plu_all_USD[i] +=m2
            else:
                mon_div_all_EUR[i] += m2
                
        elif float(df_tran_H_edu_all.currency_rk[j+1]) == float(60):
            
            if m2 > 0:
                mon_plu_all_EUR[i] +=m2
            else:
                mon_div_all_EUR[i] += m2
                
        else:
            
            if m2 > 0:
                mon_plu_all_xz[i] +=m2
            else:
                mon_div_all_xz[i] += m2
        i += 1


print(mon_div_all)
print(mon_plu_all)

print(mon_div_all_USD)
print(mon_plu_all_USD)
print(mon_div_all_EUR)
print(mon_plu_all_EUR)
print(mon_div_all_xz)
print(mon_plu_all_xz)
print()
print('col-VO\n',col_VO_all)
#cr_tran_h_edu_no = mon_h_edu_no[:]/col_VO_h_edu_no[:]
#print('cr_tran')
#print(cr_tran_h_edu_no)